In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas
import numpy
import os

import plotly.io as pio
pio.renderers.default='notebook'

#import seaborn
#from matplotlib import pyplot as plt

In [ ]:
data_path = '../examples/har_trees/data/processed/pamap2.parquet'
data = pandas.read_parquet(data_path)
data

In [ ]:
def find_runs(labels : pandas.Series):

    # Detect changes
    change_points = labels != labels.shift()
    run_ids = change_points.cumsum()
    
    # Extract runs
    def foo(g):
        #print(g.iloc[0], g.index[0], g.index[-1])
        out = pandas.Series({
            'label': g.iloc[0].activity,
            'start_time': g.index[0],
            'end_time': g.index[-1],
        })
        return out
    runs = labels.to_frame().groupby(run_ids, as_index=False).apply(foo)
    return runs

find_runs(data.loc['subject101'].activity)

In [ ]:
def add_events(fig, df):

    import plotly.express as px
    from plotly.colors import qualitative
    
    labels = df['label'].unique()
    color_pool = qualitative.Set3 + qualitative.Dark24 + qualitative.Pastel1
    label_colors = dict(zip(labels, color_pool))
    assert len(labels) <= len(color_pool)

    print(label_colors.keys())
    
    # Plot rectangles
    for _, row in df.iterrows():
        fig.add_shape(
            type='rect',
            x0=row['start_time'],
            x1=row['end_time'],
            y0=0,
            y1=1,
            xref='x',
            yref='paper',
            fillcolor=label_colors[row['label']],
            opacity=0.3,
            line_width=0,
            layer='below'
        )
    
        # Optional label annotation
        fig.add_annotation(
            x=(row['start_time'] + (row['end_time'] - row['start_time']) / 2),
            y=1.02,
            text=row['label'],
            showarrow=False,
            xref='x',
            yref='paper',
            font=dict(size=10),
        )

def time_ticks(times, every=30):
    n_times = len(times)
    start = times.min()
    end = times.max()

    def minute_second_format(t : float):
        """MM:SS"""
        return f"{int(t//60):02}:{int(t%60):02}"
    
    tick_vals = numpy.arange(start, end, every)
    tick_text = [ minute_second_format(t) for t in tick_vals]

    return tick_vals, tick_text

def plot_timeline(df,
                  data: list[str],
                  time='time',
                  label='activity',
                 ):

    import plotly.graph_objects as go

    df = df.reset_index()
    df[time] = df[time] / pandas.Timedelta(seconds=1) # convert to seconds
    df[time] -= df[time].min()
    df = df.sort_values(time)
    print(df[time].min(), df[time].max())

    fig = go.Figure()
    
    if label is not None:
        df = df.set_index(time)
        events = find_runs(df.activity)
        events = events[~events.label.isin(['transient'])]
        df = df.reset_index()
        add_events(fig, events)
    
    # Add each axis as a line
    for column in data:
        y = df[column]
        print(column)
        #print(df[time])
        fig.add_trace(go.Scatter(x=df[time], y=y,
                                 mode='lines',
                                 name=column)
                     )

        
    
    tick_vals, tick_text = time_ticks(df[time], every=60)
    
    # Customize layout
    fig.update_layout(
        #title='Tri-Axial Accelerometdf[data[0]]er Data Over Time',
        #xaxis_title='Time',
        #yaxis_title='Acceleration (g)',
        #legend_title='Axis',
        template='plotly_white',
        #hovermode='x unified'
        xaxis=dict(
            title='Elapsed Time (MM:SS)',
            tickmode='array',
            tickvals=tick_vals,
            ticktext=tick_text,
        ),
    )

    return fig

sensor_columns = ['hand_acceleration_6g_x', 'hand_acceleration_6g_y', 'hand_acceleration_6g_z']
one = data.loc['subject102']
fig = plot_timeline(one, data=sensor_columns)
fig

In [ ]:
data.activity.value_counts()

In [ ]:
one.values.shape

In [ ]:
def convert_to_raw(df: pandas.DataFrame, in_max=60.0, out_max=(2**15)-1, dtype=numpy.int16):

    missing = df.isna().any().sum() / len(df)
    print('missing values', 100*missing, '%')
    df = df.fillna(0.0)
    scaled = ((df / in_max) * out_max).clip(-out_max, out_max)
    #print(scaled)
    #print(scaled[numpy.isinf(scaled)])
    out = scaled.astype(dtype)

    return out

convert_to_raw(one[sensor_columns])

In [ ]:
import os
import subprocess
import tempfile

def process(df : pandas.DataFrame, micropython_bin='micropython'):
    __file__ = './foo.py'
    here = os.path.dirname(__file__)

    # check input
    data = numpy.ascontiguousarray(df.values)    
    assert len(data.shape) == 2, data.shape
    assert data.shape[1] == 3, data.shape
    assert data.dtype == numpy.int16, data.dtype
    assert data.flags['C_CONTIGUOUS'], data.flags

    times = df.index
    
    with tempfile.TemporaryDirectory() as temp:
        temp = ''
        in_path = os.path.join(temp, 'input.npy')
        out_path = os.path.join(temp, 'output.npy')

        # write input data
        numpy.save(in_path, data, allow_pickle=False)

        # run the program, as subprocess
        script = os.path.join(here, 'example.py')
        args = [
            micropython_bin,
            script,
            in_path,
            out_path,
        ]
        cmd = ' '.join(args)
        print('run-cmd', cmd)
        subprocess.check_output(args)

        # load output data
        assert os.path.exists(out_path)
        out = numpy.load(out_path)

        df = pandas.DataFrame(out, columns=['peak2peak', 'freqbin'])
        # XXX: ideally would use window_length and samplerate
        time_resolution = ((times.max() - times.min()) / len(df))
        df['time'] = times.min() + (time_resolution * numpy.arange(0, len(df)))
        df = df.set_index('time')
        return df

sensor_data = convert_to_raw(one[sensor_columns])
ff = process(sensor_data)
ff

In [ ]:
fig = plot_timeline(one, data=sensor_columns)
fig

In [ ]:

fig = plot_timeline(ff, data=ff.columns, label=None)
fig